In [34]:
import pandas as pd
import numpy as pp
from sklearn.utils import shuffle
import multiprocessing
from functools import partial
import string
from joblib import Parallel, delayed
import re
import copy
import spacy
from spacy.lang.en import stop_words as stop_words
import nltk
from nltk import pos_tag, word_tokenize
from nltk.stem import WordNetLemmatizer
from collections import Counter

In [35]:
wnl =WordNetLemmatizer()
df = pd.read_csv("../ml_insurance_challenge.csv")
num_cores = multiprocessing.cpu_count()

In [36]:
def replace_word(text):


    if len(text) <= 1:
        return text
    if text[0] == '-':
        text = text[1:]
    if text[-1] == "-":
        text = text[:-1]

    if "'s" not in text and "'" in text:
        text = text.replace("'","")
    return text

In [37]:
def replace_word(text):


    if len(text) <= 1:
        return text
    if text[0] == '-' or text[0]=="'":
        text = text[1:]
    if text[-1] == "-" or text[-1]=="'":
        text = text[:-1]

    if "'s" not in text and "'" in text:
        text = text.replace("'","")
    return text

In [38]:
df = df.dropna(axis=0, how='any')
colums = [('niche',0), ('description',1), ('category',2), ('business_tags',3), ('sector',4)]
category_dict = {0: 'niche', 1: 'description', 2: 'category', 3: 'business_tags', 4: 'sector'}
nlp = spacy.load('en_core_web_sm', disable=['tokenizer','parser','ner','textcat','transformer','senter','sentencizer','textcat'])

In [39]:
def replace_tag(element, tokens):

    match tokens:
        case 'NUM':
            res = any(char.isalpha() for char in element)
            if res:
                tokens = 'a'
                return tokens
            return 'n'
        case 'ADJ':
            tokens = 'a'
            return tokens
        case 'NOUN':
            tokens = 'n'
            return tokens
        case 'VERB':
            tokens = 'v'
            return tokens
        case 'ADV':
            tokens = 'r'
            return tokens
        case _:
            tokens = 'n'
            return tokens

In [40]:
from nltk.util import ngrams
 
def new_ngrams(element, n):
    for our_grams in ngrams(element, n):
            yield  " ".join(our_grams)

def new_ngrams2(element, n):
    for our_grams in ngrams(element, n):
            return  (" ".join(our_grams))


In [41]:
def lemmatize_f(tokens):

    tok = tokens
    tokens = nltk.pos_tag(word_tokenize(" ".join(tokens)), tagset='universal')

    tokens = list(map(lambda x:(replace_word(x[0].lower()),replace_tag(x[0],x[1])), tokens))

    tokens = list(filter(lambda x: (x[0],x[1]) if x[0].lower() not in stop_words.STOP_WORDS and x[0].lower() not in string.punctuation else '', tokens))


    element =  [wnl.lemmatize(token,  idx)+"_"+idx for token, idx in tokens]

    counter_element = Counter(element)
    counter_element.update(new_ngrams(element, 2))
    counter_element.update(new_ngrams(element, 3))


    return counter_element

In [42]:
def weighted_frequency(col, new_t,el=None):
    if el == None:
        el = col
    else:
        el +=col
    match new_t:
        case 'niche':
            return el+col+col+col
        case 'description':
            return el+col
        case 'category':
            return el+col+col+col+col
        case 'business_tags':
            return el+col+col+col+col+col
        case 'sector':
            el+=col+col
            return el

In [43]:
all_punctuation_except_hyphen = string.punctuation.replace("-","").replace("'","").replace("’","")
df['new_col'] = df['description']

df['new_col2'] = df['description']
df2 = copy.deepcopy(df)

In [44]:
for col, idx in colums:
    df[col] = df[col].str.replace('[{}]'.format(all_punctuation_except_hyphen), '', regex=True).str.split()
    df[col] = [list(filter(lambda x: x if ("'" not in x or "'s" in x) and (x.isnumeric() == False) else '', el)) for el in df[col]]
    df[col] = df[col].apply(lemmatize_f)


    if idx == 0:
        df['new_col'] = df.apply(lambda x: weighted_frequency(x[col], col),axis=1)
    else:
        df['new_col'] = df.apply(lambda x: weighted_frequency(x[col], col,x['new_col']),axis=1)


In [45]:
from sklearn.feature_extraction import DictVectorizer
from scipy.sparse import csr_matrix
from sklearn.feature_extraction.text import TfidfTransformer 
import numpy as np
import threading
from sklearn.metrics.pairwise import cosine_similarity



v = DictVectorizer(sparse=True)
X = v.fit_transform(df['new_col'])
feature_names = v.get_feature_names_out()
X_csr = csr_matrix(X)
tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)
tf_idf = tfidf_transformer.fit_transform(X_csr)


In [46]:
num_threads = 8
threads = []
size = X_csr.shape[0]//num_threads

global match_words_with_tf_idf_valuess
match_words_with_tf_idf_valuess={}

global match_all_words_with_tf_idf_valuess
match_all_words_with_tf_idf_valuess={}

g=[]
global zzz
zzz = 0

def get_matrix(i, start, end):
    for k in range(start, end):
        row_tfidf = X[k].toarray()[0]
        nonzero_indices = np.nonzero(row_tfidf)[0]
        g=0
        for j in nonzero_indices:
            g+=1
            a = round(tf_idf[k,j],5)
            
            match_words_with_tf_idf_valuess[(feature_names[j], k)] = a
    return match_words_with_tf_idf_valuess

for i in range(num_threads):
    start = i * size
    end = (i+1) * size if i != num_threads -1 else X_csr.shape[0]
    thread = threading.Thread(target = get_matrix, args=(i, start, end))
    threads.append(thread)
    thread.start()

for thread in threads:
    thread.join()


In [47]:
import nltk
from nltk.corpus import wordnet as wn
our_classes = pd.read_csv("../insurance_taxonomy - insurance_taxonomy.csv")
our_classes_vector = our_classes['label']

def our_classes_lemmitize(tokens):
   tokens = nltk.pos_tag(word_tokenize(" ".join(tokens)), tagset='universal')
   tokens = list(map(lambda x:(replace_word(x[0].lower()),replace_tag(x[0],x[1])), tokens))
   element =  [wnl.lemmatize(token,  idx)+"_"+idx for token, idx in tokens]
   synonyms = []
   #synonyms =  [[list(filter(lambda x: syn.lemmas()[0].name() if x.name() != el[:-2] and (x.name()+"_"+el[-1:]) in match_all_words_with_tf_idf_valuess.keys() else '', syn.lemmas())) for syn in wn.synsets(el[:-2], pos=el[-1:])] for el in element]
   synonyms=[]
   hypernyms = []
   hyponyms = []
   '''for el in element:
      for word_meanings in wn.synsets(el[:-2], pos=el[-1:]):
         word_synset =  wn.synset(word_meanings.name())
         for syn in word_meanings.lemmas():
            word_curated = syn.name()
            word_curate_space = word_curated.replace("_", " ")
            
            full_syn = syn.name() + "_" + el[-1:]
            if len(word_curate_space.split(" ")) > 1:
               word_curated = word_curate_space
               text_with_tags = nltk.pos_tag(word_tokenize(word_curate_space), tagset='universal')
               tokens_ = list(map(lambda x: x[0].lower()+"_"+replace_tag(x[0],x[1]), text_with_tags))
               full_syn = " ".join(tokens_)
            if syn.name() != el[:-2] and full_syn in match_all_words_with_tf_idf_valuess.keys():
               synonyms.append("syn_"+syn.name()+"_"+el[-1:])

         for hpe in word_meanings.hypernyms():
            hpe_synset = wn.synset(hpe.name())
            score = word_synset.path_similarity(hpe_synset)
            word_curated = hpe.name().split(".")[0]
            word_curate_space = word_curated.replace("_", " ")

            full_hpe = word_curated + "_" + el[-1:]
            full_text = "hpe_"+full_hpe+"_"+str(score)

            if len(word_curate_space.split(" ")) > 1:
               #print(word_curate_space)
               word_curated = word_curate_space
               text_with_tags = nltk.pos_tag(word_tokenize(word_curate_space), tagset='universal')
               tokens_ = list(map(lambda x: x[0].lower()+"_"+replace_tag(x[0],x[1]), text_with_tags))
               full_hpe = " ".join(tokens_)
            
            if word_curated != el[:-2] and full_hpe in match_all_words_with_tf_idf_valuess.keys() and full_text not in hypernyms:
               hypernyms.append("hpe_"+full_hpe+"_"+str(score))

         
         for hpo in word_meanings.hyponyms():
            hpo_synset = wn.synset(hpo.name())
            score = word_synset.path_similarity(hpo_synset)
            word_curated = hpe.name().split(".")[0]
            word_curate_space = word_curated.replace("_", " ")

            

            full_hpo = word_curated + "_" + el[-1:]

            if len(word_curate_space.split(" ")) > 1:
               #print(word_curate_space)
               word_curated = word_curate_space
               text_with_tags = nltk.pos_tag(word_tokenize(word_curate_space), tagset='universal')
               tokens_ = list(map(lambda x: x[0].lower()+"_"+replace_tag(x[0],x[1]), text_with_tags))
               full_hpo = " ".join(tokens_)

            full_text = "hpo_"+full_hpo+"_"+str(score)


            if word_curated != el[:-2] and full_hpo in match_all_words_with_tf_idf_valuess.keys() and full_text not in hyponyms:
               hyponyms.append("hpo_"+full_hpo+"_"+str(score))'''

   #print(hypernyms)
   #print(hyponyms)
            
   element = element +  list(new_ngrams(element,2)) +  list(new_ngrams(element,3))
   return element

In [48]:
'''listttt=[]
for wdmeanings in wn.synsets('pipeline'):
    
    for hpe in wdmeanings.hypernyms():
       
        a = wn.synset(hpe.name())
        for hpe2 in a.hypernyms():
            listttt.append(hpe2.name())
    for hpo in wdmeanings.hyponyms():
        b = wn.synset(hpo.name())
        for hpo2 in a.hyponyms():
            listttt.append(hpo2.name())
            #print(hpo2.name())
            
pipeline = wn.synset('liquid.n.01')
water = wn.synset('water.n.02')
print(water.path_similarity(pipeline))

print(listttt)'''

#print(hpe.name())
#print(hpo.name())

"listttt=[]\nfor wdmeanings in wn.synsets('pipeline'):\n    \n    for hpe in wdmeanings.hypernyms():\n       \n        a = wn.synset(hpe.name())\n        for hpe2 in a.hypernyms():\n            listttt.append(hpe2.name())\n    for hpo in wdmeanings.hyponyms():\n        b = wn.synset(hpo.name())\n        for hpo2 in a.hyponyms():\n            listttt.append(hpo2.name())\n            #print(hpo2.name())\n            \npipeline = wn.synset('liquid.n.01')\nwater = wn.synset('water.n.02')\nprint(water.path_similarity(pipeline))\n\nprint(listttt)"

In [49]:

our_classes['label'] = our_classes['label'].replace('[{}]'.format(all_punctuation_except_hyphen), '', regex=True).str.split()
our_labels_final = our_classes['label'].apply(our_classes_lemmitize)
our_classes['new_label'] = our_labels_final
our_classes['rows_number'] = range(0, len(our_labels_final))

df['rows_number'] = range(0, len(df['new_col']))


In [50]:
def match_our_words(word):
    word_split = word.split("_")
    match word_split[0]: 
        case "syn":
            return "_".join(word_split[1:])
        case "hpo" | "hpe":
            return "_".join(word_split[1:-1])
        case _:
            return word
    



In [51]:
def multiplier_word(word):
    word_split = word.split("_")
    match word_split[0]: 
        case "hpo" | "hpe":
            return float(word_split[-1])
        case _:
            return 1.0


In [54]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
embeddings_index = {}

f = open('../numberbatch-en.txt')

for line in f:
    values = line.split()
    word = values[0]
    num = word.split(" ")
    if len(num) > 1:
        print(word)
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

In [55]:
from functools import reduce
def embedding_sentence1(sentence1):
    sen1 = sentence1
    if (len(sentence1.split("_")) > 1):
        t = sen1.split("_")
        sen1 = ""
        sen1 +=t[0]
        sen1 += "".join(reduce(lambda x,y: x+"_"+y[2:], t[1:], ""))

    vec1 = embeddings_index.get(sen1, np.zeros(300))
    return vec1



def get_similarity_score(vec1, vec2):
    return cosine_similarity(vec1, vec2)[0][0]
print(df['new_col'][0])


Counter({'connection_n': 20, 'network_n': 12, 'engineering_n': 11, 'network_n connection_n': 10, 'civil_n': 9, 'civil_n engineering_n': 9, 'design_n': 8, 'service_n': 8, 'construction_n': 6, 'engineering_n construction_n': 6, 'electricity_n': 6, 'gas_n': 6, 'project_n': 6, 'optic_n': 6, 'connection_n design_n': 6, 'design_n connection_n': 6, 'connection_n optic_n': 6, 'network_n connection_n design_n': 6, 'connection_n design_n connection_n': 6, 'design_n connection_n optic_n': 6, 'engineering_n service_n': 5, 'civil_n engineering_n service_n': 5, 'heavy_n': 4, 'heavy_n civil_n': 4, 'heavy_n civil_n engineering_n': 4, 'civil_n engineering_n construction_n': 4, 'welchcivils_n': 4, 'design_v': 4, 'solution_n': 4, 'water_n': 4, 'exist_v': 4, 'civil_a': 2, 'company_n': 2, 'specialize_v': 2, 'building_n': 2, 'utility_n': 2, 'uk_n': 2, 'offer_v': 2, 'multi-utility_a': 2, 'combine_v': 2, 'fibre_a': 2, 'optic_a': 2, 'installation_n': 2, 'single_a': 2, 'contract_n': 2, 'engineer_n': 2, 'team_n'

In [65]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

bert_model = SentenceTransformer("all-MiniLM-L6-v2") 

our_classes_vector_embeddings = bert_model.encode(our_classes_vector)

texts_vector_description = []
company_texts_description = [
    f"{row['description']}"
    for index, row in df2.iterrows()
]
company_texts_business_tags = [
    f"{' '.join(row['business_tags'])}"
    for index, row in df2.iterrows()
]
company_texts_sector = [
    f"{row['sector']}"
    for index, row in df2.iterrows()
]
company_texts_category = [
    f"{row['category']}"
    for index, row in df2.iterrows()
]
company_texts_niche = [
    f"{row['niche']}"
    for index, row in df2.iterrows()
]



company_embeddings_1 = bert_model.encode(company_texts_description, batch_size=32) 
company_embeddings_2 = bert_model.encode(company_texts_business_tags, batch_size=32)  
company_embeddings_3 = bert_model.encode(company_texts_sector, batch_size=32) 
company_embeddings_4 = bert_model.encode(company_texts_category, batch_size=32) 
company_embeddings_5 = bert_model.encode(company_texts_niche, batch_size=32) 

similarity_matrix = cosine_similarity(company_embeddings_1, our_classes_vector_embeddings)
similarity_matrix += cosine_similarity(company_embeddings_2, our_classes_vector_embeddings)
similarity_matrix += cosine_similarity(company_embeddings_3, our_classes_vector_embeddings)
similarity_matrix += cosine_similarity(company_embeddings_4, our_classes_vector_embeddings)
similarity_matrix += cosine_similarity(company_embeddings_5, our_classes_vector_embeddings)

similarity_matrix /=5
print(similarity_matrix.shape)



(9455, 220)


In [57]:
'''best_matches = [np.argpartition(similarity_matrix[i], -30)[-30:] for i in range(0, len(df2))]'''

In [58]:
'''def get_best_match(word_label, var2):

    for index in best_matches[var2]:
        print(index)

        #for word_label in our_labels_final[index]:
            #print(i)
            #ff
        #sum11 = 0.0
            #print(list(df['new_col'].iloc(i)))
            #sum2+=reduce(lambda x, y: x + get_similarity_score(y[:-2], word_label[:-2]) * (match_words_with_tf_idf_valuess[(y, i)]) if get_similarity_score(y[:-2], word_label[:-2]) >=0.2 else x, df['new_col'][i], 0.0)

    return 0'''

"def get_best_match(word_label, var2):\n\n    for index in best_matches[var2]:\n        print(index)\n\n        #for word_label in our_labels_final[index]:\n            #print(i)\n            #ff\n        #sum11 = 0.0\n            #print(list(df['new_col'].iloc(i)))\n            #sum2+=reduce(lambda x, y: x + get_similarity_score(y[:-2], word_label[:-2]) * (match_words_with_tf_idf_valuess[(y, i)]) if get_similarity_score(y[:-2], word_label[:-2]) >=0.2 else x, df['new_col'][i], 0.0)\n\n    return 0"

In [63]:
from functools import reduce

vec_embeddings=[]
label_embeddings = []
MAX_TOKENS = max(df['new_col'].apply(len))

global rows_

def get_sentence_embeddings(word_list, row, opt):
    embeddings_ = []
    if opt == 1:

        sum_list_tf_idf = sum(match_words_with_tf_idf_valuess[(
            word,
            row 
        )] for word in word_list)
        embeddings_ = [
        (match_words_with_tf_idf_valuess[(
            word,
            row 
        )] * embeddings_index.get(
            word.split("_")[0] + "".join(
                reduce(
                    lambda x, y: x + "_" + y[2:] if len(word.split("_")) > 2 else "", 
                    word[:-2].split("_")[1:], 
                    ""
                )
            ),
            np.zeros(300)
        ))/ sum_list_tf_idf
            for word in word_list
        ]
    else:
        embeddings_ = [
        embeddings_index.get(
            word.split("_")[0] + "".join(
                reduce(
                    lambda x, y: x + "_" + y[2:] if len(word.split("_")) > 2 else "", 
                    word[:-2].split("_")[1:], 
                    ""
                )
            ),
            np.zeros(300)
        )
            for word in word_list
        ]


    embeddings_ = np.mean(embeddings_, axis=0)
    if (len(embeddings_)) < X.shape[1]:
        embeddings_= np.pad(embeddings_, (0, (400-len(embeddings_))), mode = 'constant')

    return embeddings_

vec_embeddings = np.array(df.apply(lambda x: get_sentence_embeddings(x['new_col'], x['rows_number'], 1), axis=1).to_list())

label_embeddings = np.array(our_classes.apply(lambda x: get_sentence_embeddings(x['new_label'], x['rows_number'], 2), axis=1).to_list())
del(match_words_with_tf_idf_valuess)
del(match_all_words_with_tf_idf_valuess)



In [67]:
np.savetxt("vec_embeddings.txt", vec_embeddings, fmt="%.12f")
np.savetxt("label_embeddings.txt", label_embeddings, fmt="%.12f")
np.savetxt("context_matrix.txt", similarity_matrix, fmt="%.12f")

: 